# Setup

In [ ]:
# %pip install ee geopandas

In [ ]:
# Follow: https://colab.research.google.com/github/google/earthengine-community/blob/master/tutorials/intro-to-python-api-guiattard/index.ipynb#scrollTo=i6WDkfesPYeV
# conda install -c conda-forge earthengine-api

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
%matplotlib inline

In [ ]:
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
import ee

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

# Extract time series

In [ ]:
import pandas as pd

def ee_array_to_df(arr, list_of_bands):
    """Transforms client-side ee.Image.getRegion array to pandas.DataFrame."""
    df = pd.DataFrame(arr)

    # Rearrange the header.
    headers = df.iloc[0]
    df = pd.DataFrame(df.values[1:], columns=headers)

    # Remove rows without data inside.
    df = df[['longitude', 'latitude', 'time', *list_of_bands]].dropna()

    # Convert the data to numeric values.
    for band in list_of_bands:
        df[band] = pd.to_numeric(df[band], errors='coerce')

    # Convert the time field into a datetime.
    df['datetime'] = pd.to_datetime(df['time'], unit='ms')

    # Keep the columns of interest.
    df = df[['time','datetime',  *list_of_bands]]

    return df

In [ ]:
fpath_hotspots = 'methane_hotspots_start_date=2021-03-01_end_date=2021-03-04.geojson'
hotspots = gpd.from_file(fpath_hotspots)

In [ ]:
start_date="2021-02-01"
end_date = "2021-04-01"

roi = ee.Geometry.Polygon(list(hotspots.iloc[1].geometry.exterior.coords))
im_coll = (ee.ImageCollection('COPERNICUS/S5P/OFFL/L3_CH4')
           .filterBounds(roi)
           .filterDate(ee.Date(start_date),ee.Date(end_date))
           .map(lambda img: img.set('date', ee.Date(img.date()).format('YYYYMMdd')))
           .sort('date'))

region_im = im_coll.getRegion(roi, 2000).getInfo()
timestamplist = (im_coll.aggregate_array('date')
                 .map(lambda d: ee.String('T').cat(ee.String(d)))
                 .getInfo())

In [ ]:
df_mth = ee_array_to_df(region_im, ['CH4_column_volume_mixing_ratio_dry_air'])
ax = df_mth.plot(x="datetime", y="CH4_column_volume_mixing_ratio_dry_air")
ax.set_ylim(1700, 2000)